## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,75.20,94,90,9.22,light rain
1,1,Punta Arenas,CL,-53.1500,-70.9167,48.20,71,0,28.77,clear sky
2,2,East London,ZA,-33.0153,27.9116,68.00,94,20,2.30,few clouds
3,3,Upernavik,GL,72.7868,-56.1549,-9.38,85,84,4.27,broken clouds
4,4,Saint-Philippe,RE,-21.3585,55.7679,86.00,54,0,3.44,light rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria (5 pt)
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                        (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,75.20,94,90,9.22,light rain
4,4,Saint-Philippe,RE,-21.3585,55.7679,86.00,54,0,3.44,light rain
8,8,Tabou,CI,4.4230,-7.3528,77.94,89,2,5.86,clear sky
11,11,Swan Hill,AU,-35.3378,143.5544,82.00,20,0,7.67,clear sky
17,17,Busselton,AU,-33.6500,115.3333,90.00,37,0,5.01,clear sky
19,19,Butaritari,KI,3.0707,172.7902,81.52,79,52,18.25,broken clouds
22,22,Hilo,US,19.7297,-155.0900,75.20,69,75,4.61,light rain
26,26,Faanui,PF,-16.4833,-151.7500,82.42,67,97,5.93,overcast clouds
28,28,Mahebourg,MU,-20.4081,57.7000,80.60,78,75,8.05,broken clouds
38,38,Rikitea,PF,-23.1203,-134.9692,79.43,73,33,10.58,scattered clouds


In [43]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().value_counts()


Unnamed: 0  City   Country  Lat    Lng    Max Temp  Humidity  Cloudiness  Wind Speed  Current Description
False       False  False    False  False  False     False     False       False       False                  152
dtype: int64

In [48]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows. (5 pt)

# Drop nulls
clean_df = clean_df.dropna()

# Drop blanks
clean_df = clean_df.loc[(city_data_df["City"] != '') &
                         (city_data_df["Country"] != '') &
                         (city_data_df["Lat"] != '') &
                         (city_data_df["Lng"] != '') &
                         (city_data_df["Max Temp"] != '') &
                         (city_data_df["Humidity"] != '') &
                         (city_data_df["Cloudiness"] != '') &
                         (city_data_df["Wind Speed"] != '') &
                         (city_data_df["Current Description"] != '')]

len(clean_df2)

152

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates. (10 pts)
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,75.20,light rain,-21.2000,-175.2000,
4,Saint-Philippe,RE,86.00,light rain,-21.3585,55.7679,
8,Tabou,CI,77.94,clear sky,4.4230,-7.3528,
11,Swan Hill,AU,82.00,clear sky,-35.3378,143.5544,
17,Busselton,AU,90.00,clear sky,-33.6500,115.3333,
19,Butaritari,KI,81.52,broken clouds,3.0707,172.7902,
22,Hilo,US,75.20,light rain,19.7297,-155.0900,
26,Faanui,PF,82.42,overcast clouds,-16.4833,-151.7500,
28,Mahebourg,MU,80.60,broken clouds,-20.4081,57.7000,
38,Rikitea,PF,79.43,scattered clouds,-23.1203,-134.9692,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
# 7. Drop the rows where there is no Hotel Name. (10 pts)
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != '']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,75.20,light rain,-21.2000,-175.2000,Keleti Beach Resort
4,Saint-Philippe,RE,86.00,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
8,Tabou,CI,77.94,clear sky,4.4230,-7.3528,hotêl doufoulougou
11,Swan Hill,AU,82.00,clear sky,-35.3378,143.5544,Lazy River Motor Inn
17,Busselton,AU,90.00,clear sky,-33.6500,115.3333,Observatory Guest House
...,...,...,...,...,...,...,...
666,Imbituba,BR,78.80,broken clouds,-28.2400,-48.6703,Praia Hotel Imbituba
672,Loandjili,CG,79.03,light rain,-4.7561,11.8578,Résidence ARCHANGE
676,Maua,BR,75.99,clear sky,-23.6678,-46.4614,Hotel Capriccio Mauá
677,Ewa Beach,US,77.00,clear sky,21.3156,-158.0072,Base Lodging


In [50]:
# 8a. Create the output File (CSV) (5pts)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [64]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} degrees Fahrenheit</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [65]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))